### loading modules and data

In [0]:
########################################################

# If running with Google Colab

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# # If using colab
# from google.colab import files
# files.upload()
# # select the 3 .py files (models, utils, data_utils)

In [0]:
########################################################

# If running with Google Colab
# Create a folder "C247" and then store the project datasets within that folder
# Check that your datasets are setup correctly

!ls "/content/gdrive/My Drive/C247" # File path

EEG_loading.ipynb	PyTorch_Experiments.ipynb  X_train_valid.npy
person_test.npy		pytorch_rnn_example.ipynb  y_test.npy
person_train_valid.npy	X_test.npy		   y_train_valid.npy


In [0]:
from models import *
from utils import *
from data_utils import *

GPU is available


In [0]:
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")
# person_train_valid = np.load("person_train_valid.npy")
# X_train_valid = np.load("X_train_valid.npy")
# y_train_valid = np.load("y_train_valid.npy")
# person_test = np.load("person_test.npy")

# Change if your directory is different

# dataset_path = './data/' # Yiming Path
dataset_path = "/content/gdrive/My Drive/C247/" 

X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### K-Fold Training and Validation (use k=1 to get results faster)

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
vote_num = 20
num_folds = 1
num_epochs = 20

for k in range(num_folds):
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='CNNGRUnet', dropout=0.6, weight_decay=1e-4)
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size)
    if aug_type != 'window':
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


RNN TYPE: CNNGRUnet
WEIGHT DECAY: 0.0001
LEARNING RATE: 0.001
fold 1
epoch: 1      time: 23.91    loss: 728.941    train acc: 0.282    val acc: 0.305
saving best model...
epoch: 2      time: 20.89    loss: 705.974    train acc: 0.344    val acc: 0.381
saving best model...
epoch: 3      time: 21.55    loss: 672.351    train acc: 0.405    val acc: 0.414
saving best model...
epoch: 4      time: 20.06    loss: 645.268    train acc: 0.442    val acc: 0.449
saving best model...
epoch: 5      time: 20.01    loss: 623.253    train acc: 0.471    val acc: 0.473
saving best model...
epoch: 6      time: 19.94    loss: 602.483    train acc: 0.494    val acc: 0.449
epoch: 7      time: 18.77    loss: 584.839    train acc: 0.514    val acc: 0.421
epoch: 8      time: 19.78    loss: 564.787    train acc: 0.537    val acc: 0.433
epoch: 9      time: 19.50    loss: 543.462    train acc: 0.558    val acc: 0.447
epoch: 10     time: 19.55    loss: 524.342    train acc: 0.578    val acc: 0.433
epoch: 11     ti

### Training and Testing

In [0]:
TestRNN(model, X_test, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.4786
